# All libraries

In [1]:
from pandas import Series, DataFrame
import statsmodels.api as sm
import pandas as pd
import nltk
import operator
import re,string
from patsy import dmatrices
%pylab inline
from nltk.corpus import stopwords
from itertools import chain
from collections import Counter
import random
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings('ignore')
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Scraper/amazon.csv', encoding = "ISO-8859-1")

In [4]:
df.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons,Advice to management
0,"""Account Manager""","Sep 9, 2018",3.0,Former Employee,Anonymous Employee,NaN,Recommends,NaN,- Fun environment and excellent team,- Proactive scenario,Almost none opportunities to grow or to move t...,NaN
1,"""All the ""bad"" things you heard about Amazon i...","Sep 9, 2018",1.0,Former Employee,Software Developer,"Seattle, WA",Doesn't Recommend,Negative Outlook,I worked at Amazon full-time (More than a year),You get to work a LOT of hours.,Work life balance is extremely bad. Work here ...,Hopeless
2,"""Amazon is giving hard time to the employees""","Sep 9, 2018",1.0,Current Employee,Fruad Prevention,Hyderabad (India),Doesn't Recommend,Negative Outlook,Becoming worst day by day,All I can say is Amazon has been changed drast...,Working at Amazon is the con,NaN
3,"""Amazon Warehouse Operative""","Sep 9, 2018",1.0,Current Employee,Anonymous Employee,NaN,Doesn't Recommend,Negative Outlook,I have been working at Amazon full-time,Weekly pay was a plus,"Treated as a number, long hours and no flexibi...",Treat staff equal
4,"""Amazon was the easiest job and I was surround...","Sep 9, 2018",5.0,Former Employee,Associate,"Essex, MD",Recommends,Positive Outlook,I worked at Amazon full-time (Less than a year),"Fun, easy, positive energy, surrounded by cool...",Long hours but youâll get used to it,NaN


In [5]:
df.shape

(25056, 12)

# Analysis of Pros and Cons

In [6]:
pros = df['Pros'].astype(str)
cons = df['Cons'].astype(str)

In [7]:
#clean punctuation and white spaces
pros_clean = pros.apply(lambda x:re.sub(r'[^\w\s]', ' ', x.lower()))
pros_clean = pros_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
pros_clean = pros_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
pros_clean = pros_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

In [8]:
#tokenizing and removing stop words
stop = set(stopwords.words('english'))
punc = string.punctuation
pros_clean = pros_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
pros_clean = pros_clean.apply(lambda x: [word for word in x if word not in punc])

In [10]:
#getting frequency distribution
all_pros = pros_clean.sum()

In [9]:
pros_dist = nltk.FreqDist(all_pros)
pros_dict = pd.DataFrame.from_dict(pros_dist,orient='index').reset_index()
pros_dict.columns = ['word','frequency']

NameError: name 'all_pros' is not defined

In [11]:
pros_dict.head()

,word,frequency
0,proactive,12
1,scenario,2
2,get,1890
3,work,9076
4,lot,1769


In [12]:
type(pros_dict)

pandas.core.frame.DataFrame

In [13]:
pros_dict.sort_values(by='frequency', ascending=0)[:20]

,word,frequency
3,work,9076
35,good,6784
41,great,5853
27,people,3794
114,benefits,3436
19,pay,3333
45,company,3190
7,amazon,3036
36,environment,2531
96,time,1931


# Bigrams

In [14]:
pro_bigrams = ngrams(all_pros, 2)
print(Counter(pro_bigrams).most_common(20))

[(('place', 'work'), 855), (('smart', 'people'), 836), (('good', 'pay'), 762), (('fast', 'paced'), 755), (('great', 'benefits'), 697), (('work', 'environment'), 696), (('work', 'culture'), 623), (('good', 'benefits'), 590), (('good', 'work'), 504), (('great', 'place'), 492), (('people', 'work'), 413), (('pay', 'good'), 395), (('work', 'life'), 386), (('learn', 'lot'), 376), (('great', 'pay'), 371), (('great', 'people'), 357), (('life', 'balance'), 352), (('company', 'work'), 332), (('decent', 'pay'), 320), (('co', 'workers'), 320)]


In [15]:
pro_trigrams = ngrams(all_pros, 3)
print(Counter(pro_trigrams).most_common(20))

[(('work', 'life', 'balance'), 344), (('great', 'place', 'work'), 282), (('fast', 'paced', 'environment'), 182), (('good', 'place', 'work'), 143), (('good', 'work', 'culture'), 124), (('great', 'company', 'work'), 121), (('good', 'work', 'environment'), 111), (('work', 'smart', 'people'), 106), (('good', 'pay', 'good'), 101), (('great', 'work', 'environment'), 96), (('day', 'work', 'week'), 93), (('good', 'work', 'life'), 92), (('4', 'day', 'work'), 90), (('great', 'people', 'work'), 90), (('best', 'place', 'work'), 89), (('good', 'working', 'environment'), 83), (('really', 'smart', 'people'), 83), (('good', 'pay', 'benefits'), 80), (('great', 'place', 'learn'), 76), (('pay', 'good', 'benefits'), 75)]


# Let's look at cons

In [16]:
cons_clean = cons.apply(lambda x:re.sub(r'[^\w\s]', ' ', x.lower()))
cons_clean = cons_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
cons_clean = cons_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
cons_clean = cons_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

#tokenizing and removing stop words
stop = set(stopwords.words('english'))
punc = string.punctuation
cons_clean = cons_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
cons_clean = cons_clean.apply(lambda x: [word for word in x if word not in punc])

#getting frequency distribution
all_cons = cons_clean.sum()

In [17]:
cons_dist = nltk.FreqDist(all_cons)
cons_dict = pd.DataFrame.from_dict(cons_dist,orient='index').reset_index()
cons_dict.columns = ['word','frequency']

In [18]:
cons_dict.sort_values(by='frequency', ascending=0)[:20]

,word,frequency
7,work,8247
28,hours,2972
80,management,2703
8,life,2597
50,time,2427
31,get,2423
9,balance,2206
27,long,2176
19,amazon,2106
99,people,2105


In [19]:
con_bigrams = ngrams(all_cons, 2)
print(Counter(con_bigrams).most_common(20))

[(('work', 'life'), 2106), (('life', 'balance'), 2043), (('long', 'hours'), 1100), (('hard', 'work'), 301), (('work', 'hard'), 257), (('fast', 'paced'), 217), (('mandatory', 'overtime'), 204), (('work', 'environment'), 198), (('short', 'breaks'), 189), (('10', 'hours'), 186), (('peak', 'season'), 173), (('place', 'work'), 168), (('low', 'pay'), 160), (('much', 'work'), 157), (('work', 'pressure'), 157), (('10', 'hour'), 150), (('working', 'amazon'), 145), (('working', 'hours'), 143), (('work', 'load'), 142), (('work', 'long'), 141)]


In [20]:
con_trigrams = ngrams(all_cons, 3)
print(Counter(con_trigrams).most_common(20))

[(('work', 'life', 'balance'), 2003), (('poor', 'work', 'life'), 109), (('work', 'long', 'hours'), 100), (('life', 'balance', 'work'), 72), (('long', 'working', 'hours'), 70), (('10', 'hour', 'shifts'), 68), (('bad', 'work', 'life'), 68), (('long', 'hours', 'work'), 65), (('â', 'work', 'life'), 52), (('long', 'hours', 'short'), 49), (('balance', 'work', 'life'), 46), (('10', 'hours', 'day'), 46), (('work', 'work', 'life'), 45), (('hours', 'short', 'breaks'), 44), (('good', 'work', 'life'), 44), (('great', 'place', 'work'), 42), (('none', 'none', 'none'), 42), (('life', 'balance', 'good'), 41), (('hours', 'work', 'life'), 41), (('long', 'work', 'hours'), 39)]


In [21]:
from gensim.models.tfidfmodel import TfidfModel
import gensim

# Let's combine both pros and cons into a column 

In [24]:
pnc = df['Pros_Cons']

In [16]:
core_values = """Customer Obsession. Leaders start with the customer and work backwards. They work vigorously to earn and keep customer trust. Although leaders pay attention to competitors, they obsess over customers. Ownership
Leaders are owners. They think long term and don’t sacrifice long-term value for short-term results. They act on behalf of the entire company, beyond just their own team. They never say that’s not my job. 

Invent and Simplify
Leaders expect and require innovation and invention from their teams and always find ways to simplify. They are externally aware, look for new ideas from everywhere, and are not limited by “not invented here". As we do new things, we accept that we may be misunderstood for long periods of time.

Are Right, A Lot
Leaders are right a lot. They have strong judgment and good instincts. They seek diverse perspectives and work to disconfirm their beliefs.

Learn and Be Curious
Leaders are never done learning and always seek to improve themselves. They are curious about new possibilities and act to explore them.

Hire and Develop the Best
Leaders raise the performance bar with every hire and promotion. They recognize exceptional talent, and willingly move them throughout the organization. Leaders develop leaders and take seriously their role in coaching others. We work on behalf of our people to invent mechanisms for development like Career Choice.

Insist on the Highest Standards
Leaders have relentlessly high standards - many people may think these standards are unreasonably high. Leaders are continually raising the bar and drive their teams to deliver high quality products, services and processes. Leaders ensure that defects do not get sent down the line and that problems are fixed so they stay fixed.

Think Big
Thinking small is a self-fulfilling prophecy. Leaders create and communicate a bold direction that inspires results. They think differently and look around corners for ways to serve customers.

Bias for Action
Speed matters in business. Many decisions and actions are reversible and do not need extensive study. We value calculated risk taking. 

Frugality
Accomplish more with less. Constraints breed resourcefulness, self-sufficiency and invention. There are no extra points for growing headcount, budget size or fixed expense.

Earn Trust
Leaders listen attentively, speak candidly, and treat others respectfully. They are vocally self-critical, even when doing so is awkward or embarrassing. Leaders do not believe their or their team’s body odor smells of perfume. They benchmark themselves and their teams against the best.

Dive Deep
Leaders operate at all levels, stay connected to the details, audit frequently, and are skeptical when metrics and anecdote differ. No task is beneath them.

Have Backbone; Disagree and Commit
Leaders are obligated to respectfully challenge decisions when they disagree, even when doing so is uncomfortable or exhausting. Leaders have conviction and are tenacious. They do not compromise for the sake of social cohesion. Once a decision is determined, they commit wholly.

Deliver Results
Leaders focus on the key inputs for their business and deliver them with the right quality and in a timely fashion. Despite setbacks, they rise to the occasion and never settle."""

In [21]:
core_values.split('\n\n')[1]

'Invent and Simplify\nLeaders expect and require innovation and invention from their teams and always find ways to simplify. They are externally aware, look for new ideas from everywhere, and are not limited by “not invented here". As we do new things, we accept that we may be misunderstood for long periods of time.'

In [ ]:
c1 = """Customer Obsession. Leaders start with the customer and work backwards. They work vigorously to earn and keep customer trust. Although leaders pay attention to competitors, they obsess over customers. Ownership
Leaders are owners. They think long term and don’t sacrifice long-term value for short-term results. They act on behalf of the entire company, beyond just their own team. They never say that’s not my job. 
"""

c2 = """Insist on the Highest Standards
Leaders have relentlessly high standards - many people may think these standards are unreasonably high. Leaders are continually raising the bar and drive their teams to deliver high quality products, services and processes. Leaders ensure that defects do not get sent down the line and that problems are fixed so they stay fixed."""


In [14]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [27]:
type(pros_clean.sum())

list

In [28]:
pros_str = ' '.join(pros_clean.sum())

In [29]:
cosine_sim(pros_str, core_values)

0.21553968492444436

In [30]:
list_pros_cons = all_pros + all_cons 

In [31]:
all_str = ' '.join(list_pros_cons)

In [32]:
cosine_sim(all_str, core_values)

0.23666609349546605

In [33]:
cosine_sim('a little bird', 'a little bird')

0.9999999999999998